Basic example of provision calculation when demands in buildings are calculated and the adjacency matrix is computed. Threshold - the value above which buildings will be marked as not meeting the specified value.

In [ ]:
%load_ext autoreload
%autoreload 2
from objectnat import get_provision,get_adjacency_matrix
import geopandas as gpd
import pandas as pd

builds = gpd.read_parquet("examples_data/spb_buildings.parquet")
services = gpd.read_parquet("examples_data/spb_services.parquet.")
adjacency_matrix = pd.read_parquet('examples_data/matrix.parquet')
builds = builds[builds['is_living']]
builds.dropna(subset=['resident_number'],inplace=True)
builds['demand'] = (builds['resident_number']*0.8).astype(int)

adjacency_matrix.index = adjacency_matrix.index.astype(int)
services.index = services.index.astype(int)
builds.index = builds.index.astype(int)

adjacency_matrix.columns = adjacency_matrix.columns.astype(int)

build_prov, services_prov, links_prov = get_provision(
    buildings=builds,
    services=services,
    adjacency_matrix=adjacency_matrix,
    threshold=10,
    city_crs=32643,
)

# Visualize data
build_prov.reset_index(drop=True,inplace=True)
services_prov.reset_index(drop=True,inplace=True)
m1 = build_prov.explore()
services_prov.explore(m=m1,color='red')
#links_prov.explore(m=m1)

Example of using `get_provision()` with the minimum amount of available data. 
- To create an `adjacency_matrix` inside, you need to provide a `nx_graph` and `weight_adjacency_matrix` - the selected weight of the edge (time_min or length_meter).
- If there is no road `graph`, as in the example below, you need to pass `city_osm_id` to load the graph from osm and local `city_crs` for calculation accuracy, as well as `weight_adjacency_matrix` to create a matrix based on the selected weight. 
- If demands in buildings are absent but buildings are resettled, provide `demand_normative` (value from 0 to 1). 
- If buildings are not resettled, `demand_normative` and `population` - the total number of residents in the area under consideration should be passed.

In [ ]:
from src.objectnat import get_provision
import geopandas as gpd

builds = gpd.read_parquet("examples_data/spb_buildings.parquet")
services = gpd.read_parquet("examples_data/spb_services.parquet")
builds = builds[builds['is_living']]


build_prov, services_prov, links_prov = get_provision(
        buildings=builds,
        services=services,
        threshold=15,
        city_osm_id=1124722,
        city_crs=32636,
        demand_normative=0.6,
        population=36000,
        weight_adjacency_matrix= "time_min",
)


m1 = build_prov.explore()
m2 = services_prov.explore(m=m1)
m3 = links_prov.explore(m=m2)
m3